# Ungraded Lab: Huber Loss

In this lab, we'll walk through how to create custom loss functions. In particular, we'll code the [Huber Loss](https://en.wikipedia.org/wiki/Huber_loss) and use that in training the model.

In [1]:
import tensorflow as tf
import numpy as np

2025-01-24 13:00:57.277172: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 13:00:57.455189: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737723657.531060  663976 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737723657.556509  663976 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 13:00:57.750298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Prepare the Data

Our dummy dataset is just a pair of arrays `xs` and `ys` defined by the relationship $y = 2x - 1$. `xs` are the inputs while `ys` are the labels.

In [2]:
xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Training the model

Let's build a simple model and train using a built-in loss function like the `mean_squared_error`.

In [3]:
model = tf.keras.Sequential(
    layers=[
        tf.keras.layers.Dense(units=1, input_shape=[1])
    ]
)

/home/mbarbaric/dev/ai/tensorflow_advanced_techniques_specialization/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1737724030.146319  663976 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5566 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [4]:
model.compile(optimizer="sgd", loss="mean_squared_error")

In [5]:
model.fit(xs, ys, epochs=500, verbose=0)

I0000 00:00:1737724060.349756  665239 service.cc:148] XLA service 0x7f2e880165f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737724060.350134  665239 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
I0000 00:00:1737724060.388850  665239 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1737724060.473111  665239 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [8]:
print(model.predict(np.array([10.0], dtype=float)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[18.976974]]


### Custom Loss

Now let's see how we can use a custom loss. We first define a function that accepts the ground truth labels (`y_true`) and model predictions (`y_pred`) as parameters. We then compute and return the loss value in the function definition.

In [13]:
def huber_loss(y_true, y_pred):
    threshold = 1
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)

In [14]:
model = tf.keras.models.Sequential(
    layers=[
        tf.keras.layers.Dense(units=1, input_shape=[1])
    ]
)

/home/mbarbaric/dev/ai/tensorflow_advanced_techniques_specialization/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer='sgd', loss=huber_loss)

In [16]:
model.fit(xs, ys, epochs=500, verbose=0)

2025-01-24 13:15:11.678225: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


In [17]:
print(model.predict(np.array([10.0], dtype=float)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
[[18.377068]]
